# Data integration, Visualization and ETL
I used 'train.csv' file where i have my dataset for training

In [142]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3c4c594765474e10870775abb04cbf2f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QoPFoBkCOcy5_z9sCxxIlro_taDDVczipnPs_wXceqeF',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3c4c594765474e10870775abb04cbf2f.get_object(Bucket='defaultikram-donotdelete-pr-ht8dbkrt3n4iv5',Key='train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

print(type(df_data_1))

<class 'pandas.core.frame.DataFrame'>


In [143]:
from pyspark.sql.types import *


mySchema = StructType([ StructField("battery_power", IntegerType(), True)\

                       ,StructField("blue", IntegerType(), True)\

                       ,StructField("clock_speed", DoubleType(), True)\

                       ,StructField("dual_sim", IntegerType(), True)\

                       ,StructField("fc", IntegerType(), True)\

                       ,StructField("four_g", IntegerType(), True)\

                       ,StructField("int_memory", IntegerType(), True)\

                       ,StructField("m_dep", DoubleType(), True)\

                       ,StructField("mobile_wt", IntegerType(), True)\

                       ,StructField("n_cores", IntegerType(), True)\

                       ,StructField("pc", IntegerType(), True)\

                       ,StructField("px_height", IntegerType(), True)\

                       ,StructField("px_width", IntegerType(), True)\

                       ,StructField("ram", IntegerType(), True)\

                       ,StructField("sc_h", IntegerType(), True)\

                       ,StructField("sc_w", IntegerType(), True)\

                       ,StructField("talk_time", IntegerType(), True)\

                       ,StructField("three_g", IntegerType(), True)\

                       ,StructField("touch_screen", IntegerType(), True)\

                       ,StructField("wifi", IntegerType(), True)\

                       ,StructField("price_range", IntegerType(), True)])

In [144]:

df = spark.createDataFrame(df_data_1,schema=mySchema)
df.show()


+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [146]:
df.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)
 |-- price_range: integer (nullable = true)



In [147]:
df.createOrReplaceTempView('Dataset')

# Initial Data Exploration
Now, I am going to implement some simple functions that i learned earlier so that i can have a view of my dataset. I will find min, mix, standard deviation and some other properties of my data columns and will explore them on different columns. I have imported data using pandas dataframe but now converted it to spark and created a view so that i can run quries on it.

In [148]:
"""
    function to find minimum of a column
"""
def findMin(columnName):
    return spark.sql("SELECT min(battery_power) as minVal from Dataset").first().minVal

In [149]:
"""
    function to find maximum of a column
"""
def findMax(columnName):
    return spark.sql('select max('+columnName+') as maxVal from Dataset').first().maxVal

In [150]:
"""
    function to find mean of a column
"""
def findMean(columnName):
    return spark.sql('select avg('+columnName+') as meanVal from Dataset').first().meanVal

In [151]:
"""
    function to find starndard deviation of a column
"""
def findstdDev(columnName):
    return spark.sql('select stddev('+columnName+') as stddev from Dataset').first().stddev

In [152]:
"""
    function to find skewness for a column
"""
def findSkew(columnName):    
    return spark.sql("""
SELECT 
    (
        1/count(*)
    ) *
    SUM (
        POWER("""+columnName+"""-%s,3)/POWER(%s,3)
    )
    as skewVal
 from Dataset
                    """ %(findMean(columnName),findstdDev(columnName))).first().skewVal


In [155]:
minVal = findMin('battery_power')
print(minVal)

501


In [156]:
findSkew('battery_power')

0.031850640034327575

In [157]:
"""
    function to find kurtosis for specific column
"""
def kurtosis(columnName):    
        return spark.sql("""
SELECT 
    (
        1/count(*)
    ) *
    SUM (
        POWER("""+columnName+"""-%s,4)/POWER(%s,4)
    )
as kut
from Dataset
                    """ %(findMean(columnName),findstdDev(columnName))).first().kut


In [158]:
kurtosis('battery_power')

1.7741403624192582

In [159]:
def correlationinMobileWeightandBatteryPower():
    return spark.sql("SELECT corr(battery_power,mobile_wt) as corrVal from Dataset").first().corrVal

In [160]:
correlationinMobileWeightandBatteryPower()

0.0018443762145708168

# Feature Creation
As i have checked and visualized data above. I would have loved to plot it using matplotlib.pyplot but as data has 21 dimensions so it is impossible to plot hyperdimension here. I have visualized data using different parameters. As i pbtained dataset from Kaggle and checked data. there was no need of big cleaning because data was not redundant or all the columns have values. There is null value in any cell. Now, it is time to create features and define model using sparkml

In [183]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


vectorAssembler = VectorAssembler(inputCols=['battery_power','blue','clock_speed','dual_sim', 'fc','four_g',
                                             'int_memory','m_dep','mobile_wt','n_cores', 'pc','px_height','px_width',
                                             'ram','sc_h','sc_w','talk_time','three_g','touch_screen','wifi'], outputCol='features')

normalizer = Normalizer(inputCol='features', outputCol='features_normalized', p=1.0)

# Creating ML Pipeline

In [184]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages =[vectorAssembler, normalizer])

model = pipeline.fit(df)

new_df = model.transform(df)

new_df.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+--------------------+--------------------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|            features| features_normalized|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+--------------------+--------------------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7| 

# Deleting useless columns
this now has become a big dataframe and i don't want that. I have created features and normalized. I have performed all of them using ML pipeline availabel in Apache spark. Now its time to delete that columns that i don't need
Let's do it.

In [185]:
df2 = new_df.drop('battery_power').drop('blue').drop('clock_speed').drop('dual_sim').drop('fc')\
.drop('four_g').drop('int_memory').drop('m_dep').drop('mobile_wt').drop('n_cores').drop('pc')\
.drop('px_height').drop('px_width').drop('ram').drop('sc_h').drop('sc_w').drop('talk_time')\
.drop('three_g').drop('touch_screen').drop('wifi')

In [186]:
df2.show()

+-----------+--------------------+--------------------+
|price_range|            features| features_normalized|
+-----------+--------------------+--------------------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|
|          3|[1821.0,0.0,1.7,0...|[0.27360829389226...|
|          0|[1954.0,0.0,0.5,1...|[0.42857456188449...|
|          0|[1445.0,1.0,0.5,0...|[0.35633260998224...|
|          0|[509.0,1.0,0.6,1....|[0.14327131477467...|
|          3|[769.0,1.0,2.9,1....|[0.12704444077317...|
|          3|[1520.0,1.0,2.2,0...|[0.22396746577865...|
|          1|[1815.0,0.0,2.8,0...|[0.37105941039375...|
|          2|[803.0,1.0,2.1,0....|[0.14541569330508...|
|          0|[1866.0,0.0,0.5,0...|[0.53989931138

# Machine Learning Algorithms

## 1. Decision Tree Classifier

In [190]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(featuresCol="features_normalized", labelCol="price_range")

model = classifier.fit(df2)

model = model.transform(df2)

model.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|price_range|            features| features_normalized|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|  [10.0,8.0,0.0,0.0]|[0.55555555555555...|       0.0|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|[0.0,7.0,40.0,365.0]|[0.0,0.0169902912...|       3.0|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|[0.0,7.0,40.0,365.0]|[0.0,0.0169902912...|       3.0|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|[0.0,7.0,40.0,365.0]|[0.0,0.0169902912...|       3.0|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|[45.0,165.0,40.0,...|[0.18,0.66,0.16,0.0]|       1.0|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|[45.0,165.0,40.0,...|[0.18,0.66,0.16,0.0]|       1.0|
|          3|[1821.

In [193]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("price_range")
    
binEval.evaluate(model) 

0.8115

In [196]:
df2.show()

+-----------+--------------------+--------------------+
|price_range|            features| features_normalized|
+-----------+--------------------+--------------------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|
|          3|[1821.0,0.0,1.7,0...|[0.27360829389226...|
|          0|[1954.0,0.0,0.5,1...|[0.42857456188449...|
|          0|[1445.0,1.0,0.5,0...|[0.35633260998224...|
|          0|[509.0,1.0,0.6,1....|[0.14327131477467...|
|          3|[769.0,1.0,2.9,1....|[0.12704444077317...|
|          3|[1520.0,1.0,2.2,0...|[0.22396746577865...|
|          1|[1815.0,0.0,2.8,0...|[0.37105941039375...|
|          2|[803.0,1.0,2.1,0....|[0.14541569330508...|
|          0|[1866.0,0.0,0.5,0...|[0.53989931138

## 2. Random Forest Classifier

In [203]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features_normalized", labelCol="price_range")

model1 = classifier.fit(df2)

model1 = model1.transform(df2)

model1.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|price_range|            features| features_normalized|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|[2.05613725543917...|[0.10280686277195...|       1.0|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|[0.56028438564035...|[0.02801421928201...|       3.0|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|[0.48607733553512...|[0.02430386677675...|       3.0|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|[0.38257816842505...|[0.01912890842125...|       3.0|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|[1.57970920595967...|[0.07898546029798...|       1.0|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|[2.17233447955945...|[0.10861672397797...|       1.0|
|          3|[1821.

In [204]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("price_range")
    
binEval.evaluate(model1) 

0.8565

## 3. Logistic Regression Classifier

In [206]:
from pyspark.ml.classification import LogisticRegression

classifier =LogisticRegression(featuresCol="features_normalized", labelCol="price_range")

model3 = classifier.fit(df2)

model3 = model3.transform(df2)

model3.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|price_range|            features| features_normalized|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|[-4.6117991971757...|[3.25847702938626...|       2.0|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|[-4.5399931399192...|[7.52007943914062...|       2.0|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|[-4.4826035856861...|[7.68929870609152...|       2.0|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|[-5.2814555859558...|[2.07710523953717...|       2.0|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|[-1.3744837438914...|[0.01698546786883...|       1.0|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|[0.74947976026659...|[0.08969699656940...|       1.0|
|          3|[1821.

In [207]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("price_range")
    
binEval.evaluate(model3) 

0.7605

# Accuracy Comparision between ML models

### Decision Tree Classifier ==> 81.15 %
### Random Forest Classifier ==> 85.65 %
### Logistic Regression ==> 76.05 %

by examining above evaluations i have found Random forest best working on this dataset and has more accuracy

# Deep Learning 
I am doing it using pytorch framework

In [209]:
!pip install torch torchvision

    100% |################################| 752.0MB 37kB/s  eta 0:00:01K    12% |###                             | 91.4MB 58.2MB/s eta 0:00:12    31% |##########                      | 237.9MB 56.7MB/s eta 0:00:10.5MB 58.4MB/s eta 0:00:09    40% |############                    | 302.1MB 47.5MB/s eta 0:00:10.3MB 55.3MB/s eta 0:00:07.9MB 65.8MB/s eta 0:00:06.8MB 66.9MB/s eta 0:00:03
    100% |################################| 6.6MB 2.1MB/s eta 0:00:01
    100% |################################| 20.2MB 900kB/s eta 0:00:01                                | 552kB 62.9MB/s eta 0:00:01
    100% |################################| 829kB 4.0MB/s eta 0:00:01
    100% |################################| 2.1MB 3.4MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.


### Again taking our initial dataset in Pandas dataframe and converting it to pytorch tensor

In [223]:
df_data_1

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
5,1859,0,0.5,1,3,0,22,0.7,164,1,...,1004,1654,1067,17,1,10,1,0,0,1
6,1821,0,1.7,0,4,1,10,0.8,139,8,...,381,1018,3220,13,8,18,1,0,1,3
7,1954,0,0.5,1,0,0,24,0.8,187,4,...,512,1149,700,16,3,5,1,1,1,0
8,1445,1,0.5,0,0,0,53,0.7,174,7,...,386,836,1099,17,1,20,1,0,0,0
9,509,1,0.6,1,2,1,9,0.1,93,5,...,1137,1224,513,19,10,12,1,0,0,0


In [224]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

In [227]:
label_df = df_data_1['price_range']
df_train_new = df_data_1.drop(labels='price_range', axis=1)

train_data = torch.tensor(df_train_new.values)
print(train_data.shape)

train_label = torch.tensor(label_df.values)
train_label

torch.Size([2000, 20])


tensor([1, 2, 2,  ..., 3, 0, 3])

In [236]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Linear(20,15)
        self.layer2 = nn.Linear(15,8)
        self.layer3 = nn.Linear(8,4)
    
    def forward(self,x):
        
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        
        return x

In [ ]:
model = Classifier()

optimizer = optim.SGD(model.parameters(), lr=0.005)

criterian = nn.CrossEntropyLoss()

train_data = train_data.float()
train_lable = train_label.float()

epoch = 200
for i in range(epoch):
    
    optimizer.zero_grad()
    output = model(train_data)
    loss = criterian(output,train_label)
    loss.backward()
    optimizer.step()
    
    print('At ',i,'/100 epoch loss is: ',loss.item())

At  0 /100 epoch loss is:  146.50144958496094
At  1 /100 epoch loss is:  1192.158203125
At  2 /100 epoch loss is:  32.88018798828125
At  3 /100 epoch loss is:  1.7925831079483032
At  4 /100 epoch loss is:  1.7575114965438843
At  5 /100 epoch loss is:  1.7265074253082275
At  6 /100 epoch loss is:  1.7000300884246826
At  7 /100 epoch loss is:  1.676580548286438
At  8 /100 epoch loss is:  1.6549211740493774
At  9 /100 epoch loss is:  1.6349304914474487
At  10 /100 epoch loss is:  1.616404414176941
At  11 /100 epoch loss is:  1.599343180656433
At  12 /100 epoch loss is:  1.5836085081100464
At  13 /100 epoch loss is:  1.5691335201263428
At  14 /100 epoch loss is:  1.5557527542114258
At  15 /100 epoch loss is:  1.5434727668762207
At  16 /100 epoch loss is:  1.5321037769317627
At  17 /100 epoch loss is:  1.521689772605896
At  18 /100 epoch loss is:  1.5120354890823364
At  19 /100 epoch loss is:  1.5032122135162354
At  20 /100 epoch loss is:  1.4950437545776367
At  21 /100 epoch loss is:  1.48